In [14]:
from loaddata import *
import pandas as pd
from random import random
from random import randint
from collections import Counter

In [5]:
df = load_data('datasets/capture20110811.pcap.netflow.labeled')
df.head()

,Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Label,src_ip,src_port,dst_ip,dst_port
0,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,135,1,Background,89.31.8.11,23929,147.32.84.229,13363
1,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,276,1,Background,147.32.84.229,13363,89.31.8.11,23929
2,2011-08-11,10:10:00.006,0.000,UDP,INT,0,1,62,1,Background,208.88.186.6,34042,147.32.84.229,13363
3,2011-08-11,10:10:00.008,0.000,UDP,INT,0,1,78,1,Background,92.118.218.77,55246,147.32.84.229,13363
4,2011-08-11,10:10:00.009,0.000,UDP,INT,0,1,72,1,Background,182.185.139.181,10223,147.32.84.229,13363


In [22]:
# Get ground truth result
# Top10 most frequent IPs connected to the host
def get_top_connected_ip(seq):
    # Count results
    count = Counter(seq)
    result = count.most_common(10)
    return result


host_ip = '147.32.84.165'
seq = load_ip_sequence(df, host_ip)
truth = get_top_connected_ip(seq)
for str in truth:
    print(str)

('193.23.181.44', 6442)
('174.128.246.102', 4101)
('174.37.196.55', 3707)
('173.236.31.226', 3410)
('184.154.89.154', 3344)
('67.19.72.206', 3224)
('46.4.36.120', 3150)
('72.20.15.61', 3111)
('147.32.80.9', 1573)
('212.117.171.138', 967)


In [7]:
def reservoir_sampling(size, data):
    reservoir = data[0: reservoir_size]

    for i in range(size, len(data)):
        prob = size / i

        if random() >= prob:
            # Generate a random int to be the replaced sample index
            index = randint(0,size-1)
            # Replace old sample
            reservoir[index] = data[i]
    
    return reservoir 

In [25]:
# Get sampling result
# Try different reservoir size
for i in [100, 1000, 2500, 5000, 7500, 10000, 20000, 30000]:
    print("********** size = %d **********" % i)
    reservoir_size = i
    
    # Reservior sampling
    result = reservoir_sampling(reservoir_size, seq.tolist())
    
    # Count results
    count = Counter(result)
    print(count.most_common(10))

********** size = 100 **********
[('184.154.89.154', 13), ('217.163.21.39', 12), ('94.100.28.114', 9), ('46.4.36.120', 7), ('212.117.171.138', 6), ('83.133.119.197', 6), ('98.139.175.225', 4), ('98.137.54.237', 3), ('209.85.143.27', 3), ('205.188.103.2', 2)]
********** size = 1000 **********
[('184.154.89.154', 157), ('46.4.36.120', 84), ('98.137.54.237', 44), ('98.139.175.225', 36), ('217.163.21.39', 35), ('147.32.80.9', 33), ('209.85.143.27', 27), ('184.82.148.44', 23), ('173.192.170.88', 20), ('184.82.148.43', 20)]
********** size = 2500 **********
[('184.154.89.154', 325), ('46.4.36.120', 244), ('147.32.80.9', 117), ('212.117.171.138', 70), ('184.82.148.43', 64), ('98.137.54.237', 64), ('173.192.170.88', 61), ('98.139.175.225', 55), ('173.236.31.226', 48), ('184.82.148.44', 47)]
********** size = 5000 **********
[('184.154.89.154', 630), ('46.4.36.120', 471), ('173.236.31.226', 241), ('147.32.80.9', 181), ('212.117.171.138', 149), ('74.6.136.244', 142), ('173.192.170.88', 104), ('6